# Análisis de la distribución de la popularidad (número de reseñas en el primer mes) #

El objetivo de este notebook es analizar la distribución del número de reseñas de los juegos en Steam

In [ ]:
import pandas as pd
import seaborn as sns
import json
import gzip
import matplotlib.pyplot as plt
import plotly.express as px
from pathlib import Path

In [ ]:
base_dir_path =  Path().resolve().parents[1]
df = pd.read_parquet(base_dir_path / "data/processed/games_info.parquet")

## Gráficos + Conclusiones

#### Histograma de la distribución de la popularidad

In [ ]:
plt.hist(x=df["recomendaciones_totales"], bins=100)
plt.yscale("log")
plt.xlabel("Numero de reseñas")
plt.ylabel("Número de juegos (log)")
plt.title("Distribución de popularidad (número de reseñas en el primer mes)")

plt.show()

#### Histogramas divididos entre juegos gratuitos y no gratuitos

In [ ]:
fig = px.histogram(df, x = "recomendaciones_totales",title="Distribución de popularidad (número de reseñas en el primer mes)",
                   labels={"x":"Número de reseñas", "y":"Número de juegos (log)"},nbins= 100,color = "free", opacity= 0.9,facet_col="free")
fig.update_yaxes(type="log")

fig.show()

In [ ]:
print(f"Número de juegos con menos de 50 reseñas en el primer mes: {df[df["recomendaciones_totales"] < 50].shape[0]}")

#### Análisis para las distintas categorías

##### Boxplot del número de reseñas para distintos géneros (los más populares)

In [ ]:
df_exp = df.explode("genres")

# Se toman los 10 géneros más populares
top_genres = df_exp["genres"].value_counts().head(10).index
top_genres

plt.Figure(figsize=(12,6))

sns.boxplot(data = df_exp[df_exp["genres"].isin(top_genres)],
            x = "genres",
            y = "recomendaciones_totales")

plt.title("Distribución de la popularidad por género en Steam")
plt.yscale("log")
plt.xticks(rotation = 45)
plt.ylabel("Número de reseñas")
plt.xlabel("Género")

plt.show()

In [ ]:
sns.violinplot(data = df_exp[df_exp["genres"].isin(top_genres)],
               x = "genres",
               y = "recomendaciones_totales")

plt.title("Distribución de la popularidad por género en Steam")
plt.yscale("log")
plt.xticks(rotation = 45)
plt.ylabel("Número de reseñas")
plt.xlabel("Género")
plt.show()


En ambos gráficos se obtienen las mismas conclusiones. La media del número de reseñas a lo largo de los distintos géneros no parece variar demasiado, además dentro de cada género la distribución sigue siendo de cola larga.

##### Histograma del número de reseñas para los géneros más populares

In [ ]:
fig, ax = plt.subplots(2,5, figsize = (15,8))

# Para convertir en lista que se pueda recorrer con 1 sólo índice
ax = ax.flatten()

for i,genre in enumerate(top_genres):
    data = df_exp[df_exp["genres"] == genre].get("recomendaciones_totales")

    ax[i].hist(data, bins=30)
    ax[i].set_title(genre)
    ax[i].set_yscale("log")

plt.tight_layout()
plt.show()

Se puede observar como todos los géneros tienen distribuciones muy similares, con gran concentracion en los números de reseñas bajos y una presencia notable de "outliers". Todo esto es coherente con la lógica del mercado de videojuegos ya que por cada género siempre hay algunos títulos que suelen destacar masivamente respecto a los demás; mientras que el grueso se suele localizar en números de reseñas bajos.

#### Conclusiones:

Se aprecia que la distribución era la que esperábamos, una de cola larga con gran concentración de juegos alrededorde números de reseñas bajos y solo unos pocos con muchas reseñas. Además parece que la distribución se mantiene en las distintas categorías (los juegos gratuitos presentan la misma distribución que los no gratuitos) y géneros.